In [1]:
import os
import pandas as pd


In [2]:
import os
os.chdir('c:\\Users\\Vigneswaran\\OneDrive - Datanetiix Solutions Inc\\Desktop\\Diesease_Prognosis')


In [3]:
import sys
import os
sys.path.append(os.path.join(os.getcwd(), 'src'))


In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    n_estimators: int
    min_samples_split: int
    min_samples_leaf: int
    random_state: int
    target_column: str

In [5]:
from diseaseprognosis.constants import *
from diseaseprognosis.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.RandomForest
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            n_estimators = params.n_estimators,
            min_samples_split = params.min_samples_split,
            random_state = params.random_state,
            min_samples_leaf = params.min_samples_leaf,
            target_column = schema.name
        )

        return model_trainer_config


In [9]:
import pandas as pd
import os
from diseaseprognosis import logger
from sklearn.ensemble import RandomForestClassifier
import joblib

In [12]:
from sklearn.ensemble import RandomForestClassifier
import joblib
import os

class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
        # Initialize Random Forest model with parameters from the config
        self.model = RandomForestClassifier(
            n_estimators=self.config.n_estimators,
            min_samples_split=self.config.min_samples_split,
            random_state=self.config.random_state
        )

    def train(self):
        # Load training and testing data
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        # Split into features and target
        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[self.config.target_column]
        test_y = test_data[self.config.target_column]

        # Train the Random Forest model
        self.model.fit(train_x, train_y)

        # Save the trained model to a file
        model_path = os.path.join(self.config.root_dir, self.config.model_name)
        joblib.dump(self.model, model_path)
        print(f"Model saved at {model_path}")

        return model_path


In [13]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-10-25 12:48:49,246: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-10-25 12:48:49,251: INFO: common: yaml file: params.yaml loaded successfully]
[2024-10-25 12:48:49,279: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-10-25 12:48:49,282: INFO: common: created directory at: artifacts]
[2024-10-25 12:48:49,285: INFO: common: created directory at: artifacts/model_trainer]
Model saved at artifacts/model_trainer\model.joblib
